In [14]:
import pickle
import torch
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import sys

# Add paths so Python can find the event_log_loader module when unpickling
sys.path.insert(0, '..')
sys.path.insert(0, '../..')

In [18]:
#Helpdesk
# data_path='../../../perturbed_data/helpdesk/redo_pert.pkl'
# result_path='../../../encoded_data/helpdesk/redo_activity.pkl'
# encoder_path='../../../encoded_data/data_encoder/helpdesk_encoder_decoder.pkl'
# properties_path='../../../encoded_data/data_encoder/helpdesk_event_log_properties.pkl'

#sepsis
# data_path='../../../perturbed_data/sepsis/redo_activity.pkl'
# result_path='../../../encoded_data/sepsis/redo_activity.pkl'
# encoder_path='../../../encoded_data/data_encoder/sepsis_encoder_decoder.pkl'
# properties_path='../../../encoded_data/data_encoder/sepsis_event_log_properties.pkl'

#BPIC17
data_path='../../../perturbed_data/BPIC17/redo_pert.pkl'
result_path='../../../encoded_data/BPIC17/redo_activity.pkl'
encoder_path='../../../encoded_data/data_encoder/BPIC17_encoder_decoder.pkl'
properties_path='../../../encoded_data/data_encoder/BPIC17_event_log_properties.pkl'

In [19]:
def encode_single_dataframe(df, encoder_decoder, case_name_col, case_id_value):
    """
    Encode a single DataFrame (prefix or suffix) into tensor format.
    
    Args:
        df: DataFrame to encode (prefix or suffix)
        encoder_decoder: TensorEncoderDecoder instance
        case_name_col: Name of the case column
        case_id_value: Value for the case ID (to ensure proper grouping)
    
    Returns:
        Tuple of (categorical_tensors, numerical_tensors) where each is a list
        of tensors with shape (1, window_size)
    """
    # Ensure the DataFrame has the case_name column for proper encoding
    df_copy = df.copy()
    if case_name_col not in df_copy.columns:
        df_copy[case_name_col] = case_id_value
    else:
        # Ensure all rows have the same case_id
        df_copy[case_name_col] = case_id_value
    
    # Encode categorical columns
    cat_tensors = []
    for col in encoder_decoder.categorical_columns:
        if col not in df_copy.columns:
            # Create zero tensor if column missing
            cat_tensors.append(torch.zeros((1, encoder_decoder.window_size), dtype=torch.long))
            continue
            
        # Get values for this column
        case_values = np.array(df_copy[[col]], dtype=object)
        # Transform using the fitted encoder
        case_values_enc = encoder_decoder.categorical_encoders[col].transform(case_values) + 1
        
        # Pad to window_size
        padded = encoder_decoder.pad_to_window_size(case_values_enc)
        # Convert to tensor and add batch dimension
        tensor = torch.tensor(padded, dtype=torch.long).squeeze(-1)  # shape: (window_size,)
        tensor = tensor.unsqueeze(0)  # shape: (1, window_size)
        cat_tensors.append(tensor)
    
    # Encode continuous columns
    num_tensors = []
    for col in encoder_decoder.continuous_columns + encoder_decoder.continuous_positive_columns:
        if col not in df_copy.columns:
            # Create zero tensor if column missing
            num_tensors.append(torch.zeros((1, encoder_decoder.window_size), dtype=torch.float32))
            continue
            
        # Get values for this column
        case_values = df_copy[[col]].values  # shape (n, 1)
        # Impute and transform
        case_values_imputed = encoder_decoder.continuous_imputers[col].transform(case_values)
        case_values_enc = encoder_decoder.continuous_encoders[col].transform(case_values_imputed)
        
        # Pad to window_size
        padded = encoder_decoder.pad_to_window_size(case_values_enc)
        # Convert to tensor and add batch dimension
        tensor = torch.tensor(padded, dtype=torch.float32).squeeze(-1)  # shape: (window_size,)
        tensor = tensor.unsqueeze(0)  # shape: (1, window_size)
        num_tensors.append(tensor)
    
    return (cat_tensors, num_tensors)

In [20]:
# Load the trained encoder_decoder
encoder_decoder = torch.load(encoder_path, weights_only=False)
#load data
data = torch.load(data_path, weights_only=False)

with open(properties_path, 'rb') as f:
    props = pickle.load(f)

# Encode all prefix/suffix pairs
encoded_data = {}
for (case_id, prefix_len), (prefix_df, suffix_df) in tqdm(data.items(), desc="Encoding data"):
    # Encode prefix
    encoded_prefix = encode_single_dataframe(
        prefix_df, encoder_decoder, props["case_name"], case_id
    )
    # Encode suffix
    encoded_suffix = encode_single_dataframe(
        suffix_df, encoder_decoder, props["case_name"], case_id
    )
    # Store encoded pair
    encoded_data[(case_id, prefix_len)] = (encoded_prefix, encoded_suffix)

print(f"Encoded {len(encoded_data)} prefix/suffix pairs")

torch.save(encoded_data, result_path)

/home/chair/henryks_students/leon_urny/Robustness-in-suffix-prediction/.venv/lib64/python3.13/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OrdinalEncoder from version 1.7.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/chair/henryks_students/leon_urny/Robustness-in-suffix-prediction/.venv/lib64/python3.13/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.7.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/chair/henryks_students/leon_urny/Robustness-in-suffix-prediction/.venv/li

Encoding data:   0%|          | 0/171611 [00:00<?, ?it/s]

Encoded 171611 prefix/suffix pairs
